In [1]:
import configparser
import pandas as pd
from sqlalchemy import create_engine

cfg = configparser.ConfigParser()
cfg.read('config.ini')
db = cfg['database']

uri = (
    f"postgresql+psycopg2://{db['user']}:{db['password']}"
    f"@{db['host']}:{db['port']}/{db['dbname']}"
)
engine = create_engine(uri)


In [2]:
sql = """
SELECT
  weather_records.municipality_id,
  municipality_name,
  station_name,
  station_code,
  postal_code,
  date,
  temperature_observed_min,
  temperature_observed_max,
  temperature_observed_avg,
  temperature_forecast_min,
  temperature_forecast_max,
  temperature_forecast_avg,
  humidity_observed_min,
  humidity_observed_max,
  humidity_observed_avg,
  humidity_forecast_min,
  humidity_forecast_max,
  humidity_forecast_avg
FROM weather_records
join municipalities on weather_records.municipality_id = municipalities.municipality_id
ORDER BY date;
"""
df = pd.read_sql(sql, engine, parse_dates=['date'])


In [3]:
print(df.shape)

(7145, 18)


In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive_output


# Dropdown options
municipalities = sorted(df["municipality_name"].unique())

# Map: label ➜ (observed_col, forecast_col)
series_map = {
    "Temperature(Minimum)":  ("temperature_observed_min",  "temperature_forecast_min"),
    "Temperature(Maximum)":  ("temperature_observed_max",  "temperature_forecast_max"),
    "Temperature(Average)":  ("temperature_observed_avg",  "temperature_forecast_avg"),
    "Relative Humidity(Minimum)":     ("humidity_observed_min",     "humidity_forecast_min"),
    "Relative Humidity(Maximum)":     ("humidity_observed_max",     "humidity_forecast_max"),
    "Relative Humidity(Average)":     ("humidity_observed_avg",     "humidity_forecast_avg"),
}

# Widgets
w_municipality   = widgets.Dropdown(options=municipalities, description="Municipality")
w_series = widgets.Dropdown(options=list(series_map.keys()), description="Series")

# Plot function
def make_plot(municipality, series_label):

    # Map the selected series label to its observed and forecast column names
    col_observed, col_forecast = series_map[series_label]
    
    # Filter dataframe for the chosen municipality and relevant columns, sort by date, drop rows with missing values
    subset = (
        df.loc[df["municipality_name"] == municipality, ["date", col_observed, col_forecast]]
          .sort_values("date")
    )
    # Initialize a Plotly figure
    fig = go.Figure()

    # Add observed data trace as a line
    fig.add_trace(go.Scatter(
        x=subset["date"], y=subset[col_observed],
        mode="lines", name="Observed data",
        line=dict(color="#1f77b4")
    ))

    # Add forecast data trace as a line
    fig.add_trace(go.Scatter(
        x=subset["date"], y=subset[col_forecast],
        mode="lines", name="Forecast data",
        line=dict(color="#7f7f7f")
    ))
    
    # Configure layout: titles, axis labels, hover mode, template, and margins
    fig.update_layout(
        title=f"{series_label} — {municipality}",
        xaxis_title="Date",
        yaxis_title=series_label.split(" • ")[0],
        hovermode="x unified",
        template="plotly_white",
        legend_title=None,
        margin=dict(l=40, r=20, t=60, b=40)
    )
    # Display the plot in the notebook
    fig.show()

# Create interactive widgets linked to the make_plot function
out = interactive_output(make_plot, {"municipality": w_municipality, "series_label": w_series})
# Arrange widgets and output vertically
widgets.VBox([widgets.HBox([w_municipality, w_series]), out])